In [3]:
#import modules
import pandas as pd
import numpy as np
import Bio
from Bio import Seq
from Bio import SeqIO
import torch
import matplotlib.pyplot as plt
import sys
from torch.utils.data import TensorDataset, DataLoader

In [4]:
#import Thermophllic proteins datasets
thermo_dict = {}
with open('/usr/lusers/achazing/thermo_proteins/ThermoDrift/data/uniprot-thermophilus.fasta') as thermo_fasta_file:  # Will close handle cleanly
    identifiers = []
    sequence = []
    for seq_record in SeqIO.parse(thermo_fasta_file, 'fasta'):  # (generator)
        identifiers.append(str(seq_record.id))
        sequence.append(str(seq_record.seq))
        thermo_dict[str(seq_record.id)] = str(seq_record.seq)

thermo_list = list(thermo_dict.items())
df_thermo = pd.DataFrame(thermo_list)
df_thermo.columns = ['protein','sequence']
df_thermo['class'] = 'Thermophillic'
print(len(df_thermo.index))
#sample 1/5 of the thermo data
df_thermo = df_thermo.sample(frac=0.20)
print(len(df_thermo.index))
print('Thermo data has been processed')


FileNotFoundError: [Errno 2] No such file or directory: '/usr/lusers/achazing/thermo_proteins/ThermoDrift/data/uniprot-thermophilus.fasta'

In [ ]:
#import Psychrophillic proteins datasets
psychro_dict = {}
with open('/usr/lusers/achazing/thermo_proteins/ThermoDrift/data/uniprot-psychrophilus.fasta') as psychro_fasta_file:  # Will close handle cleanly
    psychro_identifiers = []
    psychro_sequence = []
    for seq_record in SeqIO.parse(psychro_fasta_file, 'fasta'):  # (generator)
        psychro_identifiers.append(str(seq_record.id))
        psychro_sequence.append(str(seq_record.seq))
        psychro_dict[str(seq_record.id)] = str(seq_record.seq)

psychro_list = list(psychro_dict.items())
df_psychro = pd.DataFrame(psychro_list)
df_psychro.columns = ['protein','sequence']
df_psychro['class'] = 'Psychrophillic'

print('Psychro data has been processed')


In [5]:
#import mesophillic proteins datasets
meso_dict = {}
with open('/usr/lusers/achazing/thermo_proteins/ThermoDrift/data/uniprot-mesophilus.fasta') as meso_fasta_file:  # Will close handle cleanly
    meso_identifiers = []
    meso_sequence = []
    for seq_record in SeqIO.parse(meso_fasta_file, 'fasta'):  # (generator)
        meso_identifiers.append(str(seq_record.id))
        meso_sequence.append(str(seq_record.seq))
        meso_dict[str(seq_record.id)] = str(seq_record.seq)

meso_list = list(meso_dict.items())
df_meso = pd.DataFrame(meso_list)
df_meso.columns = ['protein','sequence']
df_meso['class'] = 'Mesophillic'

print('Meso data has been processed')


FileNotFoundError: [Errno 2] No such file or directory: '/usr/lusers/achazing/thermo_proteins/ThermoDrift/data/uniprot-mesophilus.fasta'

In [6]:
# concatenate the three dataframes
df_combine = pd.concat([df_thermo,df_meso,df_psychro]).reset_index(drop=True)


NameError: name 'df_thermo' is not defined

In [7]:
#filter out proteins that don't start with methionine and are greater than 75 aas
good_list = []
bad_list = []
sequence_list = df_combine['sequence'].tolist()
for seq in sequence_list:
    if seq.startswith('M'):
        if len(seq) > 75:
            good_list.append(seq)
        
    else:
        bad_list.append(seq)
boolean_series = df_combine.sequence.isin(good_list)
df_filter = df_combine[boolean_series]
print(len(df_combine))
print(len(df_filter))
#df_filter.head()

NameError: name 'df_combine' is not defined

In [8]:
#define a function to take in a protein sequence and output a one hot encoded version
def one_hot_encode_protein(protein_seq):
    amino_acids = "ARNDCQEGHILKMFPSTWYVUX_?-" #  the order of the one hot encoded sequences 
    aa2num= {x:i for i,x in enumerate(amino_acids)} # create a dictionary that maps amino acid to integer
    seq1hot = np.eye(len(amino_acids))[np.array([aa2num.get(res) for res in protein_seq])]
    return seq1hot

#define a function to take in a protein class and output a one hot encoded version
def one_hot_encode_class(Class):
    classes = ['Thermophillic','Mesophillic','Psychrophillic'] #  the order of the one hot encoded classes 
    class2num= {x:i for i,x in enumerate(classes)} # create a dictionary that maps class to integer
    class1hot = np.eye(len(classes))[np.array([class2num.get(Class)])]
    return class1hot

In [9]:
#define a new function to take in a protein sequence and output a one hot encoded version that is cropped and/or padded
def one_hot_encode_protein_new(protein_seq):
    amino_acids = "ARNDCQEGHILKMFPSTWYVUX_?-"  #  the order of the one hot encoded sequences 
    aa2num= {x:i for i,x in enumerate(amino_acids)} # create a dictionary that maps amino acid to integer
    if len(protein_seq) > 500:   #this argument deals with proetin sequences that are greater than 500 aas
        protein_seq = protein_seq[:500]   #crop the protein to include just the first 500 aas
        seq1hot = np.eye(len(amino_acids))[np.array([aa2num.get(res) for res in protein_seq])]   #1hotencode the cropped sequence
    if len(protein_seq) < 75:    #this argument deals with sequences that are less than 75 residues
        print('your sequences contain some less than 75 aas')
    
    else:  #if len(protein_seq) > 75 and len(protein_seq) < 500:
        protein_seq_len = len(protein_seq)   #calculate the length of the sequence
        desired_len = 500   #define the desired array length
        seq1hot_min = np.eye(len(amino_acids))[np.array([aa2num.get(res) for res in protein_seq])]   #one hot encode the sequence
        empty_res = np.zeros([(desired_len-protein_seq_len),len(amino_acids)])    #define an empty array padded with zeros
        seq1hot = np.append(seq1hot_min,empty_res,axis=0)  # append the empty array to the end of the sequence array
    return seq1hot


In [10]:
#create a list of one hot encoded protein sequences
seq_list = df_filter['sequence'].tolist()
hotencode_seq_array = np.empty((1, len(amino_acids)))
num = 0
for seq in seq_list:
    seq_array = one_hot_encode_protein_new(seq)
    #print(seq_array.shape)
    hotencode_seq_array = np.concatenate([hotencode_seq_array,seq_array])
    num=num+1
    print(num)

NameError: name 'df_filter' is not defined

In [11]:
#create a list of one hot encoded  classes
class_list = df_filter['class'].tolist()
hotencode_class_list = []
hotencode_class_array = np.empty((1, len(classes)))
for c in class_list:
    #maybe use .tolist()
    hotencode_class_list.append(one_hot_encode_class(c))
    c_array = one_hot_encode_class(c)
    hotencode_class_array = np.concatenate([hotencode_class_array,c_array])

NameError: name 'df_filter' is not defined

In [14]:
#load the arrays into tensors and save them for training and testing the CNN model
my_x = hotencode_seq_array
my_y = hotencode_class_array

tensor_x = torch.Tensor(my_x) # transform to torch tensor
tensor_y = torch.Tensor(my_y)  # transform to torch tensor

torch.save(tensor_x, 'tensor_x.pt') #download tensor to share for training
torch.save(tensor_y, 'tensor_y.pt') #download tensor to share for training